In [187]:
import praw
import pandas as pd
import datetime
import xlwt
from xlwt import Workbook
import requests
import urllib
from tqdm import tqdm
import time

In [188]:
def get_date(submission):
    time = submission.created
    return datetime.datetime.fromtimestamp(time)

In [189]:
def save_image(url):
    
    file_name = url.split("/")[-1]
    res = requests.get(url, stream = True)

    # Creating an images folder if it doens't already exist
    if not os.path.exists('images'):
       os.makedirs('images')

    urllib.request.urlretrieve(url, "images/" + file_name)

In [195]:
reddit_read_only = praw.Reddit(
    client_id='JzbofoB1-Xrh3gbv--LVTQ',
    client_secret='v3R81WmJnSJJ8WWdjn4sZvRcerIVGg',
    user_agent='Ukraine Scraper'
)
# qwerrtty423499 username
# KVKb6AqeabAG2PD password
# cveodyiphyxqccujnc@tmmbt.com email
# v3R81WmJnSJJ8WWdjn4sZvRcerIVGg secret

subreddit = reddit_read_only.subreddit("memes")

# posts = subreddit.hot(limit=10)
limit = 100
posts = subreddit.top(time_filter='year', limit=limit)
# Scraping the top posts of the current month




posts_dict = {"Title": [], "Author" : [], "Timestamp" : [], "Post Text": [],
            "ID": [], "Score": [],
            "Total Comments": [], "Image URL": []
            }

with tqdm(total=limit, bar_format='{l_bar}{bar:50}{r_bar}{bar:-10b}') as progress_bar:
    for post in posts:
        # URL of each post
        # ensures posts without images are ignored
        if (".jpg" in post.url or ".png" in post.url or ".tiff" in post.url or ".gif" in post.url):
            # image is valid
            posts_dict["Image URL"].append(str(post.url))
            save_image(str(post.url))

        else:
            continue 

        # Title of each post
        posts_dict["Title"].append(post.title)

        # Author of each post
        posts_dict["Author"].append(str(post.author))

        # Time post was created
        posts_dict["Timestamp"].append(datetime.datetime.fromtimestamp(post.created).strftime("%m/%d/%Y, %H:%M:%S"))

        # Text inside a post
        posts_dict["Post Text"].append(post.selftext)

        # Unique ID of each post
        posts_dict["ID"].append(post.id)

        # The score of a post
        posts_dict["Score"].append(post.score)

        # Total number of comments inside the post
        posts_dict["Total Comments"].append(post.num_comments)
        
        progress_bar.update()


# Displaying the data in a pandas dataframe (for testing purposes)
# pd.set_option('display.max_rows', 100)
# pd.set_option('max_colwidth', None)

# top_posts = pd.DataFrame(posts_dict)
# top_posts



 93%|██████████████████████████████████████████████▌   | 93/100 [02:05<00:09,  1.35s/it]                                           


,Title,Author,Timestamp,Post Text,ID,Score,Total Comments,Image URL
0,thank god,ludicrosity548,"05/13/2022, 08:34:41",,uoqq32,233570,1145,https://i.redd.it/djw0xedkm8z81.gif
1,Ukraine got CHAD Volodymyr Zelensky.🇺🇦🇺🇦🇺🇦,I_AM_BATMAN_69,"02/25/2022, 12:31:37",,t19inj,225363,2766,https://i.redd.it/p6hpt00hl0k81.jpg
2,Wish her luck boys,Onetimeguy8,"07/11/2022, 11:02:57",,vwkj6c,196156,1066,https://i.redd.it/fnbkdh8fwza91.gif
3,The Darwin Incident,nechdoesntno,"05/08/2022, 09:44:07",,ul20xa,174033,1492,https://i.redd.it/otvjol1ea9y81.gif
4,How could this have happened?,ihavebirb,"03/24/2022, 13:31:36",,tmpu06,162080,1219,https://i.redd.it/fk21zj23adp81.gif
5,shit just went from unlimited power to i'm too weak,ludicrosity548,"04/26/2022, 05:28:31",,uc8k2b,159499,561,https://i.redd.it/xe9zjldh9uv81.gif
6,Is it really that glorious?,Dansydemansy,"01/31/2022, 14:13:36",,sh9drx,159196,1350,https://i.redd.it/mff8fvjvo2f81.gif
7,Still waiting,GoKaroZ,"04/11/2022, 16:27:40",,u1gpa0,154158,544,https://i.redd.it/o50e3tawlys81.gif
8,I miss Rick Moranis,davetowers646,"06/08/2022, 09:17:18",,v7pg6w,152632,532,https://i.redd.it/z2b6x1xxde491.gif
9,Context: the livestream got taken down yesterday,deguligni,"07/11/2022, 05:45:10",,vwehqp,149516,3297,https://i.redd.it/665x45t5uwa91.png


In [197]:
wb = Workbook()
sheet1 = wb.add_sheet('Reddit Data 1')

# column, row

# Writing the table headers
i = 0
for key in posts_dict:
    sheet1.write(0, i, key)
    i+=1

# Writing the table data
for x in range(1, len(posts_dict["Title"])):
    i = 0
    for key in posts_dict:
        sheet1.write(x, i, posts_dict[key][x])
        i+=1

# Saving the document
wb.save('meme_data.xls')


In [154]:
# Loop with a progress bar
for i in trange(100):
    time.sleep(0.01)

  0%|          | 0/100 [00:00<?, ?it/s]